In [120]:
from datasets import load_dataset

In [121]:
dataset = load_dataset('ms_marco', 'v1.1')

In [122]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 10047
    })
    train: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 82326
    })
    test: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 9650
    })
})

In [123]:
dataset_train = load_dataset('ms_marco', 'v1.1', split='train')

In [124]:
dataset_train

Dataset({
    features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
    num_rows: 82326
})

In [125]:
type(dataset_train)

datasets.arrow_dataset.Dataset

In [126]:
def preprocess(example):
    positive_passages=[]
    for p in enumerate(example['passages']["is_selected"]):
        if p[1]==1:
            positive_passages.append(example["passages"]["passage_text"][p[0]])
    negative_passages = []
    for p in enumerate(example['passages']["is_selected"]):
        if p[1]==0:
            negative_passages.append(example["passages"]["passage_text"][p[0]])
    if (len(positive_passages)>0 and len(negative_passages)>=5):
        positive = positive_passages[0]
        negatives = negative_passages[:5]
        return {
            "query": example["query"],
            "positive": positive,
            "negatives": negatives
        }
    else:
        return {"query": None, "positive": None, "negatives": None}

In [127]:
processed_data_train = dataset_train.map(preprocess, remove_columns=dataset_train.column_names)

In [128]:
processed_data_train = processed_data_train.filter(lambda x: x['query'] is not None and x['positive'] is not None)

In [129]:
contrastive_pairs_train = []
for item in processed_data_train:
    query = item["query"]
    positive = item["positive"]
    negatives = item["negatives"]
    contrastive_pairs_train.append({
        "anchor": query,
        "positive": positive,
        "negatives": negatives
    })

In [130]:
len(contrastive_pairs_train)

74538

In [131]:
from torch.utils.data import DataLoader

In [132]:
class ContrastiveDataset:
    def __init__(self, pairs):
        self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        item = self.pairs[idx]
        return item["anchor"], item["positive"], item["negatives"]

In [133]:
contrastive_dataset_train = ContrastiveDataset(contrastive_pairs_train[0:100])

In [134]:
data_loader_train = DataLoader(contrastive_dataset_train, batch_size=16, shuffle=True)

In [135]:
len(data_loader_train)

7

In [136]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [137]:
model = AutoModel.from_pretrained("bert-base-uncased")

In [138]:
from peft import LoraConfig, get_peft_model

In [139]:
lora_config = LoraConfig(
    task_type= "FEATURE_EXTRACTION"
)

In [140]:
lora_model = get_peft_model(model, lora_config)

In [141]:
lora_model.print_trainable_parameters()

trainable params: 294,912 || all params: 109,777,152 || trainable%: 0.2686


In [142]:
import torch
import torch.nn as nn
import torch.optim as optim

In [143]:
class HyperbolicEmbeddingLayer(nn.Module):
    def __init__(self, embed_dim):
        super(HyperbolicEmbeddingLayer, self).__init__()
        self.embed_dim = embed_dim
        self.fc = nn.Linear(768, embed_dim)

    def forward(self, x):
        embedding = self.fc(x)
        return embedding / torch.norm(embedding, dim=-1, keepdim=True)

In [144]:
def lorentzian_distance(x, y):
    
    dot_product = torch.sum(x * y, dim=-1)
    norm_x = torch.norm(x, dim=-1)
    norm_y = torch.norm(y, dim=-1)

    distance = torch.acosh(-dot_product + torch.sqrt((1 + norm_x**2) * (1 + norm_y**2)))
    return distance

In [145]:
def exterior_angle(x_space, y_space, c):
    norm_x_space = torch.norm(x_space, p=2, dim=-1)
    norm_y_space = torch.norm(y_space, p=2, dim=-1)
    x_time = torch.sqrt(1/c + norm_x_space**2)
    y_time = torch.sqrt(1/c + norm_y_space**2)
    dot_product = torch.sum(x_space * y_space, dim=-1)
    lorentz_inner_product =  dot_product - x_time * y_time
    numerator = y_time + x_time * c * lorentz_inner_product
    denominator = norm_x_space * torch.sqrt((c * lorentz_inner_product)**2 - 1)
    ext_angle = torch.acos(numerator / denominator)
    return ext_angle

In [146]:
def entailment_loss(x, y, c=1, K=0.1):

    aperture = torch.asin(2 * K / (c * torch.norm(x, p=2, dim=-1))) 
    
    ext_angle = exterior_angle(x_space=x,y_space=y,c=c)
    
    loss = torch.max(torch.zeros_like(ext_angle), ext_angle - aperture)
    return loss.mean()

In [147]:
def compute_laplacian(similarity_matrix):
    similarity_matrix = similarity_matrix.float()

    degree_matrix = torch.diag(torch.sum(similarity_matrix, dim=1))

    laplacian_matrix = degree_matrix - similarity_matrix

    return laplacian_matrix

In [148]:
def harmonic_distance(laplacian_matrix):
    laplacian_pseudo_inv = torch.pinverse(laplacian_matrix)
    harmonic_distances = []
    for anchor_idx in range(0, len(laplacian_matrix), 7):  
      
        anchor_node=torch.zeros(len(laplacian_matrix), dtype=torch.float32, device=device)
        anchor_node[anchor_idx] = 1

        distances = []
        for i in range(7-1):
            node=torch.zeros(len(laplacian_matrix), dtype=torch.float32, device=device)
            node[anchor_idx+(i+1)] = 1
            diff = anchor_node - node
            dist = torch.matmul(torch.matmul(diff.T, laplacian_pseudo_inv), diff)
            distances.append(dist)
        
        harmonic_distances.append(torch.stack(distances))
    return(harmonic_distances)

In [149]:
def harmonic_loss(distances):
    logits=-torch.stack(distances, dim=0)
    labels = torch.zeros(logits.size(0), dtype=torch.long, device=logits.device)
    loss = torch.nn.CrossEntropyLoss()(logits, labels)
    return loss

In [150]:
optimizer = torch.optim.AdamW(lora_model.parameters(), lr=5e-5)

In [151]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [152]:
print(lora_model)

PeftModelForFeatureExtraction(
  (base_model): LoraModel(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Identity()
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=8, bias=False)
                  )
                  (lora_B): Modu

In [153]:
embed_dim=128

In [154]:
hyperbolic_model= HyperbolicEmbeddingLayer(embed_dim)

In [155]:
import torch.nn.functional as F

In [156]:
dataset_val = load_dataset('ms_marco', 'v1.1', split='validation')

In [157]:
processed_data_val = dataset_val.map(preprocess, remove_columns=dataset_val.column_names)

In [158]:
processed_data_val = processed_data_val.filter(lambda x: x['query'] is not None and x['positive'] is not None)

In [159]:
contrastive_pairs_val = []
for item in processed_data_val:
    query = item["query"]
    positive = item["positive"]
    negatives = item["negatives"]
    contrastive_pairs_val.append({
        "anchor": query,
        "positive": positive,
        "negatives": negatives
    })

In [162]:
contrastive_dataset_val = ContrastiveDataset(contrastive_pairs_val[0:100])

In [163]:
data_loader_val = DataLoader(contrastive_dataset_val, batch_size=16, shuffle=True)

In [164]:
len(data_loader_val)

7

In [165]:
lora_model = lora_model.to(device)
hyperbolic_model = hyperbolic_model.to(device)

In [166]:
def evaluate_mrr(model1, model2, data_loader_val):
    model1.eval() 
    model2.eval()
    
    total_rr = 0.0
    num_queries = 0

    with torch.no_grad():  
        for batch in data_loader_val:
            anchor_text = batch[0]
            positive_text = batch[1]
            negative_texts = batch[2]

            anchor_input = tokenizer(anchor_text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
            positive_input = tokenizer(positive_text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)

            anchor_embedding = model2(model1(**anchor_input).last_hidden_state[:, 0, :])
            positive_embedding = model2(model1(**positive_input).last_hidden_state[:, 0, :])
            negative_embedding = [model2(model1(**tokenizer(neg, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)).last_hidden_state[:, 0, :]) for neg in negative_texts]

            batch_embeddings = []
   
            for i in range(len(anchor_embedding)): 
                sample_embeddings = torch.cat(
                    [anchor_embedding[i:i+1],  
                    positive_embedding[i:i+1],  
                    torch.stack([neg[i] for neg in negative_embedding])], 
                    dim=0
                )
                batch_embeddings.append(sample_embeddings)
            
            all_embeddings = torch.cat(batch_embeddings, dim=0)
            sim=F.cosine_similarity(all_embeddings.unsqueeze(1), all_embeddings.unsqueeze(0), dim=2)
            laplacian = compute_laplacian(sim)
            distances=harmonic_distance(laplacian)
            all_similarities=-torch.stack(distances, dim=0)

            sorted_similarities, sorted_indices = torch.sort(all_similarities, dim=1, descending=True)

            # Find the rank of the first relevant (positive) document
            positive_rank = (sorted_indices == 0).nonzero(as_tuple=True)[1] + 1  # +1 to make rank 1-based
            total_rr += torch.sum(1.0 / positive_rank.float()).item()  # Reciprocal rank
            num_queries += len(positive_rank)

    mrr = total_rr / num_queries
    return mrr

In [170]:
import os
save_dir ="/dss/dsshome1/07/ra65bex2/srawat"
import time
epoch_metrics = []
num_epochs=15

In [171]:
for epoch in range(num_epochs):
    start_time = time.time()
    lora_model.train() 
    hyperbolic_model.train()

    total_loss = 0.0
    entailment_loss_total=0.0
    contrastive_loss_total=0.0
    for batch in data_loader_train:
     
        anchor_texts = batch[0]
        positive_texts = batch[1]
        negative_texts = batch[2]
       
        anchor_inputs = tokenizer(anchor_texts, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
        positive_inputs = tokenizer(positive_texts, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
    
        anchor_embedding = hyperbolic_model(lora_model(**anchor_inputs).last_hidden_state[:, 0, :])
        positive_embedding = hyperbolic_model(lora_model(**positive_inputs).last_hidden_state[:, 0, :])
        negative_embedding = [hyperbolic_model(lora_model(**tokenizer(neg, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)).last_hidden_state[:, 0, :]) for neg in negative_texts]

        batch_embeddings = []
     
        for i in range(len(anchor_embedding)): 
            sample_embeddings = torch.cat(
                [anchor_embedding[i:i+1],  
                positive_embedding[i:i+1], 
                torch.stack([neg[i] for neg in negative_embedding])], 
                dim=0
            )
            batch_embeddings.append(sample_embeddings)
        all_embeddings = torch.cat(batch_embeddings, dim=0)
        sim=F.cosine_similarity(all_embeddings.unsqueeze(1), all_embeddings.unsqueeze(0), dim=2)
        laplacian = compute_laplacian(sim)
        distances=harmonic_distance(laplacian)
 
        contrastive_loss_value = harmonic_loss(distances)

        entailment_loss_value = entailment_loss(anchor_embedding, positive_embedding)
        
        loss = contrastive_loss_value + 0.1*entailment_loss_value

        optimizer.zero_grad()  
        loss.backward()  
        optimizer.step() 
        
        total_loss += loss.item()
        entailment_loss_total+=entailment_loss_value.item()
        contrastive_loss_total+=contrastive_loss_value.item()
    print(f"EPOCH {epoch+1}:")
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(data_loader_train)}")
    print(f"Epoch {epoch+1}/{num_epochs}, Contrastive Loss: {contrastive_loss_total / len(data_loader_train)}")
    print(f"Epoch {epoch+1}/{num_epochs}, Entailment Loss: {entailment_loss_total / len(data_loader_train)}")
    mrr_validation = evaluate_mrr(model1=lora_model, model2=hyperbolic_model, data_loader_val=data_loader_val)
    #mrr_train = evaluate_mrr(lora_model, data_loader_train, lorentzian_distance)
    #print(f"Mean Reciprocal Rank (MRR) for training set: {mrr_train:.4f}")
    print(f"Mean Reciprocal Rank (MRR) for validation set: {mrr_validation:.4f}")
    end_time = time.time()
    print(f"Epoch {epoch+1} took {(end_time - start_time) / 60:.4f} minutes.")
    print(f"\n")
    epoch_metrics.append({
    'epoch': epoch + 1,
    'training_loss': total_loss / len(data_loader_train),
    'mrr_validation': mrr_validation,
    'time_taken_minutes': (end_time - start_time) / 60
    })

EPOCH 1:
Epoch 1/15, Loss: 1.987798810005188
Epoch 1/15, Contrastive Loss: 1.7914708001273019
Epoch 1/15, Entailment Loss: 1.9632800306592668
Mean Reciprocal Rank (MRR) for validation set: 0.4243
Epoch 1 took 0.2146 minutes.


EPOCH 2:
Epoch 2/15, Loss: 1.987417527607509
Epoch 2/15, Contrastive Loss: 1.7913611105510168
Epoch 2/15, Entailment Loss: 1.9605641875948225
Mean Reciprocal Rank (MRR) for validation set: 0.4307
Epoch 2 took 0.2961 minutes.


EPOCH 3:
Epoch 3/15, Loss: 1.9855119500841414
Epoch 3/15, Contrastive Loss: 1.7906102282660348
Epoch 3/15, Entailment Loss: 1.94901704788208
Mean Reciprocal Rank (MRR) for validation set: 0.4095
Epoch 3 took 0.5264 minutes.


EPOCH 4:
Epoch 4/15, Loss: 1.989059601511274
Epoch 4/15, Contrastive Loss: 1.7922971078327723
Epoch 4/15, Entailment Loss: 1.9676249027252197
Mean Reciprocal Rank (MRR) for validation set: 0.3857
Epoch 4 took 0.6127 minutes.


EPOCH 5:
Epoch 5/15, Loss: 1.9894168717520577
Epoch 5/15, Contrastive Loss: 1.793243357113429

In [172]:
import json
with open(save_dir + '/combined_epoch_metrics.json', 'w') as f:
    json.dump(epoch_metrics, f)